# ACTION RECOGNITION

By Rajat Jain

In [ ]:
from PIL import Image
from skimage.feature import hog
import glob
import cv2
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import LeaveOneOut
import glob
import os
import numpy as np
import time
from keras.models import Sequential
from keras.layers import Flatten,Dense,Lambda, Convolution2D, Cropping2D, Dropout
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import Adam
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")




# Preprocessing the image

Function to crop the image according to ground truth 

In [37]:
def crop_image(imgpath,gt1,gt2,gt3,gt4):
    imm = Image.open(imgpath, "r")
    return imm.crop((int(gt1),int(gt2),int(gt1)+int(gt3),int(gt2)+int(gt4)))


Reading ground truth file to get dimensions and labels

In [38]:
def read_gt(path):
    file = open(path,"r")
    return file.read().split()


Cropping Image and getting labels. Creating new folder same as original but with cropped images and labels

In [39]:

path=os.getcwd()+"/ucf_sports_actions/ucf action"

def files_cwd(dir):
    r = []
    for dirs in os.listdir(dir):
        r.append(dirs)
    return r


In [40]:
def makeFolders(path):
    if not os.path.isdir(path):
            os.makedirs(path)

In [41]:

def resizeImages(image,resX,resY):
    return cv2.resize(image,(resX,resY))
    
    

Getting all sports action in a list

In [42]:
actions=files_cwd(path)
print (actions)

['Kicking-Front', 'Swing-SideAngle', 'Golf-Swing-Back', 'Run-Side', 'Golf-Swing-Front', 'Swing-Bench', 'Kicking-Side', 'Lifting', 'Diving-Side', 'Walk-Front', 'Riding-Horse', 'SkateBoarding-Front', 'Golf-Swing-Side']


Creating Dictionary of action

In [54]:
labels={'Swing-SideAngle':1,'Kicking-Front':0, 'Golf-Swing-Back':2, 'Run-Side':3, 'Golf-Swing-Front':4, 'Swing-Bench':5, 'Kicking-Side':6, 'Lifting':7, 'Diving-Side':8, 'Walk-Front':9, 'Riding-Horse':10, 'SkateBoarding-Front':11, 'Golf-Swing-Side':12
       }



Below code will extract all images -> Cropping all images according to GT -> Resize all images to 64*128 -> Putting it in jpeg folder
>> If gt is not present and jpeg folder is present than only performing resizing 

In [ ]:
for i in actions:
    inActionPath=path+"/"+i # in action
    folders_actions=files_cwd(inActionPath)
    for j in folders_actions:
        images_path=inActionPath+"/"+j #inside 001
        #gt_path=inActionPath+"/"+j+"/gt" #inside gt
        files=files_cwd(images_path)# getting all images
        if "jpeg" not in files:
            makeFolders(images_path+"/jpeg")
            item=".jpg"
            if any(item in s for s in files):
                for f in files:
                    if os.path.isdir(images_path+"/gt"):
                        if f.endswith(".jpg"):
                            image=images_path+"/"+f
                            gt_path=images_path+"/gt/"+f.split(".")[0]+".tif.txt"
                            readGt=read_gt(gt_path)
                            croppedImage=crop_image(image,readGt[0],readGt[1],readGt[2],readGt[3])
                            croppedImage.save(image)
                            cvim=cv2.imread(image)
                            reim=resizeImages(cvim,64,128)
                            cv2.imwrite(images_path+"/jpeg/"+f, reim)
                    else:
                        print("Gt do not exist for this "+images_path)
                    
            else:
                print("Images not available in "+images_path)
        else:
            jpegPath=images_path+"/jpeg"
            jpgs=files_cwd(jpegPath)
            for x in jpgs:
                if ".jpg" in x:
                    im=cv2.imread(jpegPath+"/"+x)
                    
                    reim=resizeImages(im,64,128)
                    cv2.imwrite(jpegPath+"/"+x, reim)
                     
        
        

For train_y and test_y value : Extracting sports from image path and getting its values from labels dictionary

In [55]:
def getY_train(im):
    b=os.path.split(os.path.dirname(im))[-2]
    c=os.path.dirname(b)
    d=os.path.basename(c)
    return labels[d]

# Models - Classifiers

SVM   :    First Normalizing train_x and test_x and later applying svm

In [120]:
from sklearn.metrics import classification_report
def svmModel(train_x,test_x,train_y,test_y):
        print(len(test_x))
        print(len(train_x))
        print(len(train_y))
        print(len(test_y))

        X = np.array(train_x).astype(np.float64)                        
        X_scaler = StandardScaler().fit(X)
        scaled_X = X_scaler.transform(X)
        
        test_X = np.array(test_x).astype(np.float64)                        
        X_scaler_test = StandardScaler().fit(test_X)
        scaled_X_test = X_scaler_test.transform(test_X)

        rand_state = np.random.randint(0, 100)
        X_train, X_test, y_train, y_test = scaled_X,scaled_X_test,train_y,test_y
        
        print('Feature vector length:', len(X_train[0]))

        svc = LinearSVC()
      
        t=time.time()
        svc.fit(X_train, y_train)
        t2 = time.time()
       
        
        accuracy=round(svc.score(X_test, y_test), 4)
        print('Test Accuracy of SVC = ', accuracy)
        
        t=time.time()

        n_predict = 10
        print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
        print('For these',n_predict, 'labels: ', y_test[0:n_predict])
        t2 = time.time()
        print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')
        
        y_pred = clf.predict(X_test)
        sensitivity=classification_report(y_test, y_pred)
        val_label=labels
        val_label = {y: x for x, y in val_label.items()}

        print("sensitivity for ",val_label[y_test[0]])
        print(sensitivity)
        return accuracy

    

Random Forest 

In [121]:
def random_forest(X_train, X_test, y_train, y_test):
    
#     X = np.array(X_train).astype(np.float64)                        
#     X_scaler = StandardScaler().fit(X)
#     X_train = X_scaler.transform(X)
    
#     X = np.array(X_test).astype(np.float64)                        
#     X_scaler = StandardScaler().fit(X)
#     X_test = X_scaler.transform(X)

    
    clf = RandomForestClassifier(max_depth=7, random_state=0)
    t=time.time()
    clf.fit(X_train, y_train)
    t2 = time.time()
    print(round(t2-t, 2), 'Seconds to train random forest...')
    accuracy = round(clf.score(X_test, y_test), 4)
    print('Test Accuracy of Random Forest = ', accuracy)
    y_pred = clf.predict(X_test)
    sensitivity=classification_report(y_test, y_pred)
    val_label=labels
    val_label = {y: x for x, y in val_label.items()}

    print("sensitivity for ",val_label[y_test[0]])
    print(sensitivity)
    return accuracy


Keras Model

In [108]:
from keras.layers import Activation
import keras.backend as K

def get_categorical_accuracy_keras(y_true, y_pred):
    return K.mean(K.equal(K.argmax(y_true, axis=1), K.argmax(y_pred, axis=1)))
def kerasModel(train_x,test_x,train_y,test_y):
        model = Sequential()
        
        model.add(Convolution2D(24, 5, 5,subsample=(2,2),activation="relu",input_shape=(128,64,3)))
        
        model.add(Convolution2D(36, 5, 5,subsample=(2,2),activation="relu"))
        
        model.add(Convolution2D(48, 5, 5,subsample=(2,2),activation="relu"))
        
        model.add(Convolution2D(64, 3, 3,subsample=(2,2), activation="relu"))
               
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(64, activation="relu"))

        model.add(Dense(32, activation="relu"))

        model.add(Dense(1, activation="relu"))
        model.add(Activation('softmax'))
        sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='mse',optimizer=sgd,metrics=['accuracy'])

        model.fit(train_x, train_y,epochs=5)
       
        #model.compile(loss='mse', optimizer=Adam(lr=0.01, decay=1e-6),metrics=[get_categorical_accuracy_keras])

        #model.fit(train_x,train_y, shuffle=True, nb_epoch=5)   

        score = model.evaluate(test_x, test_y ,verbose=0)
        
        print("Accuracy: %.2f%%" % (score[1]*100))
        return score[1]*100


Extracting Features : This is for all sports actions
>> 1. Hog Features
>> 2. Spatial Features : Resizing Images : Segregate object with background
>> 3. Hist Features in different channels

In [59]:
def extract_features(hog_feat=True,spatial_feat=True,hist_feat=True):
    train_x=[]
    train_y=[]
    path=os.getcwd()+"/ucf_sports_actions/ucf action"
   
    for value,label in enumerate(labels):
        images=glob.glob(path+"/"+label+"/**/jpeg/*.jpg")
        
        for im in images:
            img=cv2.imread(im)
            hog_feature=[]
            feat_x=[]
            if hog_feat==True:
                for channel in range(img.shape[2]):
                    hog_feature.append(hog(img[:,:,channel],orientations=9,pixels_per_cell=(8,8),cells_per_block=(2,2),transform_sqrt=True, visualise=False,feature_vector=True))

                hog_feature = np.ravel(hog_feature)
                feat_x.append(hog_feature.ravel())
            
            if spatial_feat == True:
                spatial_features = cv2.resize(img, (32,32)).ravel() 
                feat_x.append(spatial_features)
            if hist_feat == True:
                
                channel1_hist = np.histogram(img[:,:,0], bins=32, range=(0,256))
                channel2_hist = np.histogram(img[:,:,1], bins=32, range=(0,256))
                channel3_hist = np.histogram(img[:,:,2], bins=32, range=(0,256))
               
                hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
                feat_x.append(hist_features)
            train_x.append(np.concatenate(feat_x))
            train_y.append(value)
            
    print(len(train_x))
    print(len(train_y))
    return train_x,train_y

SVM : Train Test Split 
>>>In Below code, I am  doing train test split >>test split 20%

In [123]:
train_x,train_y=extract_features(True,True,True)

X = np.array(train_x).astype(np.float64)                        

X_scaler = StandardScaler().fit(X)

scaled_X = X_scaler.transform(X)

rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, train_y, test_size=0.2, random_state=rand_state)
print('Feature vector length:', len(X_train[0]))
svc = LinearSVC()
svc.fit(X_train, y_train)
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))


/home/rajat/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


8488
8488
Feature vector length: 14508
Test Accuracy of SVC =  0.987


# Leave One Out

Extracting Features : This function is created for using in leave one out function (its same as above extract feature but in this we are passing images for which we want to extract features)
>> 1. Hog Features
>> 2. Spatial Features : Resizing Images : Segregate object with background
>> 3. Hist Features in different channels

In [60]:
def extract_features_LeaveOneOut(images,hog_feat,spatial_feat,hist_feat):
    train_x=[]
    train_y=[]
    path=os.getcwd()+"/ucf_sports_actions/ucf action"
   
    for im in images:
        img=cv2.imread(im)
        hog_feature=[]
        feat_x=[]
        
        if hog_feat==True:
            for channel in range(img.shape[2]):
                hog_feature.append(hog(img[:,:,channel],orientations=9,pixels_per_cell=(8,8),cells_per_block=(2,2),transform_sqrt=True, visualise=False,feature_vector=True))

            hog_feature = np.ravel(hog_feature)
            feat_x.append(hog_feature)
            
        if spatial_feat == True:
            spatial_features = cv2.resize(img, (32,32)).ravel() 
            feat_x.append(spatial_features)
        if hist_feat == True:
            
            channel1_hist = np.histogram(img[:,:,0], bins=32, range=(0,256))
            channel2_hist = np.histogram(img[:,:,1], bins=32, range=(0,256))
            channel3_hist = np.histogram(img[:,:,2], bins=32, range=(0,256))
            
            hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
            feat_x.append(hist_features)
        
        train_x.append(np.concatenate(feat_x))
        
        b=os.path.split(os.path.dirname(im))[-2]

        c=os.path.dirname(b)
        d=os.path.basename(c)
        train_y.append(labels[d])
               
    print(len(train_x))
    print(len(train_y))
    return train_x,train_y
            

Leave one out Function : 
>> Logic -> Reading one folder of sports action clases iteratively and putting it on test, rest on training.
>>
>> For SVM and Random Forest, I am extracting features and than sending it to svm or random forest
>> For Keras Model , I am reading all images and passing it to model 

In [61]:
def data_leave_one_out():
    images_list=[]
    index_list=[]
    len_subfolders=[]
    path=os.getcwd()+"/ucf_sports_actions/ucf action"
    images=glob.glob(path+"/**"+"/**/jpeg/*.jpg")
    for value,label in enumerate(labels):
        actionPath=path+"/"+label
        listdir=files_cwd(actionPath)
        for ld in listdir:
            test_x_img=glob.glob(path+"/"+label+"/"+ld+"/jpeg/*.jpg")
            images_list+=test_x_img
            index_list.append(len(test_x_img))
        len_subfolders.append(len(listdir))
    return images_list,index_list,len_subfolders

In [62]:
def keras_LOO():
    path=os.getcwd()+"/ucf_sports_actions/ucf action"
    images=glob.glob(path+"/**"+"/**/jpeg/*.jpg")
    accDict={}
    num_action_infolder={}
    path=os.getcwd()+"/ucf_sports_actions/ucf action"
    val_label=labels
    val_label = {y: x for x, y in val_label.items()}

    for i in labels.keys():
        num_action_infolder[labels[i]]=len(files_cwd(path+"/"+i))       

    for value,label in enumerate(labels):
        actionPath=path+"/"+label
        listdir=files_cwd(actionPath)
        
        for ld in listdir:
            test_x_img=glob.glob(path+"/"+label+"/"+ld+"/jpeg/*.jpg")
            if(len(test_x_img)==0):
                continue
            train_x_img=[]
            print("Label Running >>"+ label)
            print("Folder Running >>"+ ld)
            for i in images:
                if i not in test_x_img:
                    train_x_img.append(i)
            train_x=[]
            train_y=[]
            test_y=[]
            test_x=[]
            for x in train_x_img:
                train_y.append(getY_train(x))
            for y in test_x_img:
                test_y.append(getY_train(y))
            for i in train_x_img:
                train_x.append(cv2.imread(i))
            for j in test_x_img:
                test_x.append(cv2.imread(j))
            acc=kerasModel(np.array(train_x),np.array(test_x),np.array(train_y),np.array(test_y))
            if test_y[0] not in accDict.keys():
                
                accDict[test_y[0]] = acc
            else:
                accDict[test_y[0]] += acc  
    for i in accDict:
        print("Accuracy of label "+val_label[i]+" is ")
        print(accDict[i]/num_action_infolder[i])

              
            




    


Extracting Feature Separately and than using it in leave one out code

In [63]:
images_list,index_list,len_subfolders=data_leave_one_out()
print("Index list is ")
print(len(index_list))
feature_list,value_list=extract_features_LeaveOneOut(images_list,True,True,True)
print(len(feature_list))
print(len(value_list))     
X = np.array(feature_list).astype(np.float64)                        
X_scaler = StandardScaler().fit(X)
features = X_scaler.transform(X)
features=features.tolist()

Index list is 
150


/home/rajat/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


8488
8488
8488
8488


In [119]:
def leave_one_out(model):
    if model=="keras":
        keras_LOO()
    else:
        
        train_x=[]
        train_y=[]
        test_x=[]
        test_y=[]
        accDict={}
        
        num_action_infolder={}
        path=os.getcwd()+"/ucf_sports_actions/ucf action"
        val_label=labels
        val_label = {y: x for x, y in val_label.items()}
        
        for i in labels.keys():
            num_action_infolder[labels[i]]=len(files_cwd(path+"/"+i))       
            
        
        
        b=0
        for j in range(150):#As there are 150 folders
            
            if(index_list[j]==0):
                continue
            
            test_x=features[b:b+index_list[j]]#0-60 #60-120
            
            if len(test_x)==0:
                continue
            
            test_y=value_list[b:b+index_list[j]]#0-60 #60-120
            
            train_x=features[:b]+features[b+index_list[j]:]#0+60-end 0-60 
            
            train_y=value_list[:b]+value_list[b+index_list[j]:]#0+60-end
            
            b+=index_list[j]#b=j
            
            if model=="svm":
                acc=svmModel(train_x,test_x,train_y,test_y)    
            elif model=="random_forest":
                acc=random_forest(train_x,test_x,train_y,test_y)    
            print(test_y[0])
            if test_y[0] not in accDict.keys():
                
                accDict[test_y[0]] = acc
                
            else:
                accDict[test_y[0]] += acc 
                
                
            
        for i in accDict:
            print("Accuracy of label "+val_label[i]+" is ")
            print(accDict[i]/num_action_infolder[i])
            
    
    
    

In [122]:
leave_one_out("random_forest")





6.48 Seconds to train random forest...
Test Accuracy of Random Forest =  0.9091


/home/rajat/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


sensitivity for  Diving-Side
             precision    recall  f1-score   support

          1       0.00      0.00      0.00         0
          5       0.00      0.00      0.00         0
          8       1.00      0.91      0.95        55
          9       0.00      0.00      0.00         0

avg / total       1.00      0.91      0.95        55

8
6.72 Seconds to train random forest...
Test Accuracy of Random Forest =  0.4545


/home/rajat/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


sensitivity for  Diving-Side
             precision    recall  f1-score   support

          3       0.00      0.00      0.00         0
          5       0.00      0.00      0.00         0
          8       1.00      0.45      0.62        55
          9       0.00      0.00      0.00         0
         11       0.00      0.00      0.00         0

avg / total       1.00      0.45      0.62        55

8
6.69 Seconds to train random forest...
Test Accuracy of Random Forest =  0.7273


...

/home/rajat/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


sensitivity for  Golf-Swing-Front
             precision    recall  f1-score   support

          2       0.00      0.00      0.00         0
          3       0.00      0.00      0.00         0
          4       1.00      0.05      0.10        60
          9       0.00      0.00      0.00         0
         11       0.00      0.00      0.00         0

avg / total       1.00      0.05      0.10        60

4
6.1 Seconds to train random forest...
Test Accuracy of Random Forest =  0.0833


/home/rajat/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


sensitivity for  Golf-Swing-Front
             precision    recall  f1-score   support

          1       0.00      0.00      0.00         0
          3       0.00      0.00      0.00         0
          4       1.00      0.08      0.15        60
          5       0.00      0.00      0.00         0
          9       0.00      0.00      0.00         0

avg / total       1.00      0.08      0.15        60

4
6.1 Seconds to train random forest...
Test Accuracy of Random Forest =  0.1333
sensitivity for  Golf-Swing-Front
             precision    recall  f1-score   support

          4       1.00      0.13      0.24        60
          9       0.00      0.00      0.00         0
         12       0.00      0.00      0.00         0

avg / total       1.00      0.13      0.24        60

4
Accuracy of label Kicking-Front is 
0.0
Accuracy of label Swing-SideAngle is 
0.807176923077
Accuracy of label Golf-Swing-Back is 
0.31666
Accuracy of label Run-Side is 
0.3006
Accuracy of label Golf-Swing-F

/home/rajat/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
